In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, transforms

from matplotlib import pyplot as plt
from IPython.display import clear_output

import os
from PIL import Image

In [2]:
import time

In [9]:
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
folder_path = 'Img_for_video'

# Получите список файлов в папке
image_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]

# Инициализируйте список для хранения массивов изображений
image_arrays = []
i = 0
# Загрузите каждое изображение, преобразуйте его в массив и добавьте в список
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    with Image.open(image_path) as img:
        image_array = np.array(img)
        image_arrays.append(image_array)
    i += 1
    if i % 1000 == 0:
        print(f'Загрузилось {i} картинок')
# Преобразуйте список массивов в один numpy массив
orig_array = np.array(image_arrays)

print(orig_array.shape)

Загрузилось 1000 картинок
Загрузилось 2000 картинок
(2177, 420, 420, 3)


In [18]:
class OrigDataset(Dataset):
    def __init__(self, x):
        self.orig_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((400, 400))
        ])
        self.x = x
         
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        x = self.orig_transform(self.x[idx])
        return x

In [19]:
dataset =OrigDataset(orig_array)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = False)
org_array = None

In [7]:
def ConvBlock(in_channels, out_channels, p):
        block = nn.Sequential(
            #nn.LayerNorm(torch.size),
            #nn.BatchNorm2d(in_channels),
            nn.Dropout2d(p = p),
            nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 1),
            nn.ReLU(),
            nn.Dropout2d(p = 0.1),
            #nn.Dropout2d(p = 0.1),
            #nn.LayerNorm(torch.size),
            #nn.BatchNorm2d(out_channels),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            #nn.BatchNorm2d(out_channels),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 5, padding = 2),
            nn.ReLU(),
            #nn.LayerNorm(torch.size),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout2d(p = 0.1),
            #nn.LayerNorm(torch.size),
            #nn.BatchNorm2d(out_channels),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
            nn.ReLU()
        )
        return block
    
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        ch_size = [3, 32, 64, 128, 256, 512]
        self.cd1 = ConvBlock(ch_size[0], ch_size[1], 0) #200
        self.mp1 = nn.MaxPool2d(2)
        self.ln1 = nn.LayerNorm([ch_size[1], 200, 200])
        self.cd2 = ConvBlock(ch_size[1], ch_size[2], 0.1) #100
        self.mp2 = nn.MaxPool2d(2)
        self.ln2 = nn.LayerNorm([ch_size[2], 100, 100])
        self.cd3 = ConvBlock(ch_size[2], ch_size[3], 0.1) #50
        self.mp3 = nn.MaxPool2d(2)
        self.ln3 = nn.LayerNorm([ch_size[3], 50, 50])
        self.cd4 = ConvBlock(ch_size[3], ch_size[4], 0.1) #25
        self.mp4 = nn.MaxPool2d(2)
        self.ln4 = nn.LayerNorm([ch_size[4], 25, 25])
        self.cn = ConvBlock(ch_size[4], ch_size[4], 0.1)

        self.ln5 = nn.LayerNorm([ch_size[4], 25, 25])
        self.up1 = nn.Upsample(scale_factor = 2)
        self.cu1 = ConvBlock(ch_size[4], ch_size[3], 0.1) #50
        self.ln6 = nn.LayerNorm([ch_size[3], 50, 50])
        self.up2 = nn.Upsample(scale_factor = 2)
        self.cu2 = ConvBlock(ch_size[3], ch_size[2], 0.1) #100
        self.ln7 = nn.LayerNorm([ch_size[2], 100, 100])
        self.up3 = nn.Upsample(scale_factor = 2)
        self.cu3 = ConvBlock(ch_size[2], ch_size[1], 0.1) #200
        self.ln8 = nn.LayerNorm([ch_size[1], 200, 200])
        self.up4 = nn.Upsample(scale_factor = 2)
        self.cu4 = ConvBlock(ch_size[1], ch_size[0], 0.1) #400

    def forward(self, x):
        x = self.ln1(self.mp1(self.cd1(x)))
        x = self.ln2(self.mp2(self.cd2(x)))
        x = self.ln3(self.mp3(self.cd3(x)))
        x = self.ln4(self.mp4(self.cd4(x)))
        x = self.ln5(self.cn(x))
        x = self.ln6(self.up1(self.cu1(x)))
        x = self.ln7(self.up2(self.cu2(x)))
        x = self.ln8(self.up3(self.cu3(x)))
        x = self.up4(self.cu4(x))

        return(x)

In [10]:
md = Model().to(device)
md.load_state_dict(torch.load('model.pth'))
md.eval()

Model(
  (cd1): Sequential(
    (0): Dropout2d(p=0, inplace=False)
    (1): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
    (3): Dropout2d(p=0.1, inplace=False)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Dropout(p=0.1, inplace=False)
    (7): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (8): ReLU()
    (9): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Dropout(p=0.1, inplace=False)
    (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Dropout2d(p=0.1, inplace=False)
    (15): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU()
  )
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (ln1): LayerNorm((32, 200, 200), eps=1e-05, elementwise_affine=True)
  (cd2): Sequential(
    (0): Dropout2d(p=0.1, inplace=False)
    (1): Conv2d(

In [9]:
model_parameters = filter(lambda p: p.requires_grad, md.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [11]:
print(params)

223996675


In [12]:
def Images_Process(batch, model):
    batch = batch.to(device)
    with torch.no_grad():
        y = model(batch)
    
    y = y.detach().to('cpu').numpy() #возможно не надо detach
    y = y.transpose((0, 2, 3, 1))
    y[y < 0] = 0
    y = (y * 255).astype(np.uint8)

    return y

In [20]:
start_time = time.time()
for i, batch in enumerate(dataloader):
    images = Images_Process(batch, md)
    for j, img_numpy in enumerate(images):
        img = Image.fromarray(img_numpy)
        img.save(os.path.join('Img_Video',f'{i * batch_size + j + 1}.png'))
    if i % 2 == 0:
        print(f'Прошло времени: {time.time() - start_time}')
        print(f'Создалось изображений: {(i + 1) * batch_size}')
        print(f'Изображений в секунду: {((i + 1) * batch_size)/(time.time() - start_time)}')

C:\Users\maks0\anaconda3\envs\cuda1\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Прошло времени: 0.8206720352172852
Создалось изображений: 16
Изображений в секунду: 19.49621689712354
Прошло времени: 1.8520641326904297
Создалось изображений: 48
Изображений в секунду: 25.917029088119133
Прошло времени: 2.8465659618377686
Создалось изображений: 80
Изображений в секунду: 28.104038716303375
Прошло времени: 3.827570915222168
Создалось изображений: 112
Изображений в секунду: 29.261378164040902
Прошло времени: 4.866671323776245
Создалось изображений: 144
Изображений в секунду: 29.589012780970922
Прошло времени: 5.844676733016968
Создалось изображений: 176
Изображений в секунду: 30.107718210218575
Прошло времени: 6.840279817581177
Создалось изображений: 208
Изображений в секунду: 30.408112759567175
Прошло времени: 7.861059904098511
Создалось изображений: 240
Изображений в секунду: 30.53023420860481
Прошло времени: 8.83216643333435
Создалось изображений: 272
Изображений в секунду: 30.79652110873024
Прошло времени: 9.749875545501709
Создалось изображений: 304
Изображений в се